<a href="https://colab.research.google.com/github/Ashindustry007/Code-References-and-Boilerplates---DL/blob/main/Building_a_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Models API**

There are three ways to create Keras models:

* The Sequential model, which is very straightforward (a simple list of layers), but is limited to single-input, single-output stacks of layers (as the name gives away).

* The Functional API, which is an easy-to-use, fully-featured API that supports arbitrary model architectures. For most people and most use cases, this is what you should be using. This is the Keras "industry strength" model.

* Model subclassing, where you implement everything from scratch on your own. Use this if you have complex, out-of-the-box research use cases.

https://keras.io/api/models/

##Keras - Sequential Model

In [ ]:
#Creating by passing a list of layers to the sequential constructor. 

model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

In [ ]:
#Creating a sequential model incrementally via the add() method.

model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

#add to a model is like append to a list.

In [ ]:
#pop method to remove model layer. Sequential model acts very much like a python list.
model.pop()

In [ ]:
#it accepts name argument
model = keras.Sequential(name="my_sequential")
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))

Parallel Computing With Multiple GPUs

https://colab.research.google.com/github/keras-team/keras-io/blob/master/guides/ipynb/distributed_training.ipynb

Feature extraction

Once a sequential model is built, it behaves like a Functional API model. So every layer has it's own input and output attribute. 

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

#extracting feature from only one layer
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)

#extracting feature from all the intermediate layers
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

Transfer Learning

Transfer learning consists of freezing the bottom layers in a model and only training the top layer.

In [ ]:
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# 1st way
# Freeze all layers except the last one.
for layer in base_model.layers[:-1]:
  layer.trainable = False

# 2nd way
# Freeze the base model
base_model.trainable = False
# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

##Keras - Functional Model

The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.



In [ ]:
# initializing a graph network with 3 layers
# (input: 784-dimensional vectors) ↧ [Dense (64 units, relu activation)] ↧ [Dense (64 units, relu activation)] ↧ [Dense (10 units, softmax activation)] ↧ (output: logits of a probability distribution over 10 classes)

# 1st step - creating input node
inputs = keras.Input(shape=(784,)) #for normal inputs
img_inputs = keras.Input(shape=(32, 32, 3)) #for image inputs, just an example. Not to be included in this code.

# 2nd step - creating the layers in-between
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)

# 3rd step - creating the last output layer
outputs = layers.Dense(10)(x)

# Last step involves creating model by specifying its input and outputs in the graph of layers
model = keras.Model(inputs=inputs, outputs=outputs, name="name_of_model")

In [ ]:
# To visualise the model architecture
model.summary() #first type
keras.utils.plot_model(model, "name.png") #second type
keras.utils.plot_model(model, "name.png", show_shapes=True) #third type

In [ ]:
# example with an end-to-end auto-encoder model

encoder_input = keras.Input(shape=(28, 28, 1), name="original_img")
x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x) #16 as output

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

decoder_input = keras.Input(shape=(16,), name="encoded_img")
x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation="relu")(x) #28, 28, 1 as output

decoder = keras.Model(decoder_input, decoder_output, name="decoder")
decoder.summary()

# You can treat any model as if it were a layer by invoking it on an Input or on the output of another layer. 
# By calling a model you aren't just reusing the architecture of the model, you're also reusing its weights.

autoencoder_input = keras.Input(shape=(28, 28, 1), name="img")
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = keras.Model(autoencoder_input, decoded_img, name="autoencoder")
autoencoder.summary()

# Decoding architecture is strictly symmetrical to the encoding architecture, hence input shape is equal to output shape.
